This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [203]:
import os
import openai
import random
os.environ["OPENAI_API_KEY"] = input("ENTER API KEY :")
os.environ["OPENAI_API_BASE"] = input("ENTER API BASE :")

from langchain.llms import OpenAI
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_base = os.environ["OPENAI_API_BASE"]

### Step 1: Setting Up the Python Application

- Initialize a Python Project: Create a new Python project, setting up a virtual environment and installing necessary packages like LangChain, a suitable LLM library (e.g., OpenAI's GPT), and a vector database package compatible with Python (e.g., ChromaDB or LanceDB). If you don't wish to create your files from scratch, starter files are available in the workspace on the next page as an application skeleton.

In [204]:
## Launch and Install script
# conda create -n realestate_agent python=3.10.11
# eval "$(conda shell.bash hook)"
# eval conda activate realestate_agent
#conda install jupyter
# cd /home/kenneth/Documents/Projects/realestate_agent
# jupyter-notebook

#!pip install -r requirements

### Step 2: Generating Real Estate Listings

- Generate real estate listings using a Large Language Model. Generate at least 10 listings This can involve creating prompts for the LLM to produce descriptions of various properties. An example of a listing might be:

In [205]:
example_listing = """Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
Property: House 
Pool: In-ground
Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a solar heated pool and vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."""

In [206]:
listing_list = [example_listing]
for k in range(20):
    prompt = """Please create annother example listing that varies on type (house, condo), size, location, quality, price. Here are ##Examples: {} ##New Example:""".format('\n####\n'.join(random.sample(listing_list, k=min(5, len(listing_list)))))
    chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}])
    listing_list.append(chat_completion.choices[0].message.content)

In [207]:
# Write listing to file
with open("listings.txt", "w", encoding="utf-8") as f:
    f.write("\n#######\n".join(listing_list))

In [208]:
# Load Listing from file
with open("listings.txt", "r", encoding="utf-8") as f:
    listing_list = f.read().strip().split("\n#######\n")

In [209]:
[print('\n#######\n'+i) for i in listing_list[1:4]]


#######
Neighborhood: Beachfront Condos
Price: $1,200,000
Bedrooms: 2
Bathrooms: 2.5
Property: Condo 
View: Oceanfront 
Size: 1,800 sqft

Description: Immerse yourself in luxury living with this stunning beachfront condo in the heart of Beachfront Condos. This 2-bedroom, 2.5-bathroom unit features floor-to-ceiling windows offering breathtaking ocean views from every room. The modern kitchen is equipped with high-end appliances and sleek countertops, perfect for entertaining guests. Enjoy the sunrise and sunset from your private balcony overlooking the sandy beach below. With access to a rooftop pool, gym, and concierge services, this beachfront condo offers the ultimate in comfort and convenience.

Neighborhood Description: Beachfront Condos is a prestigious waterfront community with direct access to pristine sandy beaches and a vibrant boardwalk lined with shops and restaurants. Residents can enjoy beach volleyball, water sports, and sunset yoga sessions just steps from their front d

[None, None, None]

### Step 3: Storing Listings in a Vector Database

- Vector Database Setup: Initialize and configure ChromaDB or a similar vector database to store real estate listings.
- Generating and Storing Embeddings: Convert the LLM-generated listings into suitable embeddings that capture the semantic content of each listing, and store these embeddings in the vector database.

In [210]:
# Run to clear old collections
# client_db.delete_collection("real_estate_listings")

In [211]:
import chromadb
from sentence_transformers import SentenceTransformer


client_db = chromadb.Client()
#client_db.delete_collection("real_estate_listings")
collection = client_db.get_or_create_collection("real_estate_listings")

embedder = SentenceTransformer('all-mpnet-base-v2')

embeddings = embedder.encode(listing_list).tolist()

ids = [f"listing-{i}" for i in range(len(listing_list))]

collection.add(
    ids=ids,
    documents=listing_list,
    embeddings=embeddings
)




Failed to send telemetry event client_start: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event collection_add: capture() takes 1 positional argument but 3 were given


In [212]:
query_text = "Condo with balcony in city"
query_embedding = embedder.encode([query_text]).tolist()

results = collection.query(
    query_embeddings=query_embedding,
    n_results=2,
    include=['documents', 'distances']
)

In [213]:
for doc, dist in zip(results['documents'][0], results['distances'][0]):
    print(f"Match (distance: {dist:.4f}): {doc}")

Match (distance: 0.6728): Neighborhood: Urban Skyline Lofts
Price: $900,000
Bedrooms: 2
Bathrooms: 2
Property: Condo
View: City skyline
Size: 1,500 sqft

Description: Enjoy modern city living in this sleek urban loft located in the bustling Urban Skyline Lofts community. This 2-bedroom, 2-bathroom condo features floor-to-ceiling windows offering panoramic views of the city skyline, creating a contemporary and vibrant living space. The open concept layout, industrial accents, and high-end appliances in the kitchen make this loft perfect for entertaining. Step out onto the private balcony to soak in the bustling city life below. With access to a rooftop terrace, fitness center, and concierge services, this condo offers a luxurious urban lifestyle.

Neighborhood Description: Urban Skyline Lofts is a trendy and cosmopolitan community with access to hip cafes, art galleries, and creative spaces. Residents can explore nearby cultural attractions, trendy boutiques, and vibrant nightlife spots

### Step 4: Building the User Preference Interface

- Collect buyer preferences, such as the number of bedrooms, bathrooms, location, and other specific requirements from a set of questions or telling the buyer to enter their preferences in natural language. You can hard-code the buyer preferences in questions and answers, or collect them interactively however you'd like:

- Buyer Preference Parsing: Implement logic to interpret and structure these preferences for querying the vector database.

In [214]:
def ask_questions():
    questions = [
        "Please, tell me what you are looking for in a property, such as number of bedrooms, bathrooms, location, and other specific requirements, and I will try to find you a match!",
        "Is there anything else very important about the property I should consider, such as architecture or features?",
        "Is there anything else very important about the neighborhood I should consider?"
    ]
    
    answers = []
    for q in questions:
        ans = input(q + " ")
        answers.append(ans)
    
    buyer_wants = " ".join(answers)

    return buyer_wants





In [215]:
buyer_wants = ask_questions()
print(buyer_wants)

Please, tell me what you are looking for in a property, such as number of bedrooms, bathrooms, location, and other specific requirements, and I will try to find you a match!  condo with a balcony in a city centre
Is there anything else very important about the property I should consider, such as architecture or features?  modern kitchen
Is there anything else very important about the neighborhood I should consider?  lots of coffee shops for writing


condo with a balcony in a city centre modern kitchen lots of coffee shops for writing


In [216]:
buyer_wants

'condo with a balcony in a city centre modern kitchen lots of coffee shops for writing'

### Step 5: Searching Based on Preferences

- Semantic Search Implementation: Use the structured buyer preferences to perform a semantic search on the vector database, retrieving listings that most closely match the user's requirements.
- Listing Retrieval Logic: Fine-tune the retrieval algorithm to ensure that the most relevant listings are selected based on the semantic closeness to the buyer’s preferences.

In [217]:
def get_property_picks_listingdb(buyer_wants, collection, embedder):

    prompt = """
    You are an AI assistant. Your job is to extract structured buyer preferences from this input.
    
    Input:
    "{}"
    
    Extract as JSON in this format:
    {{
      "size": string,
      "property_type": string,
      "important_features": [string, string, string],
      "amenities": [string, ...],
      "transportation": string,
      "urban_level": string,
      "neighborhood_vibes": string,
      "architectural style": string,
      "pool": string
    }}
    """.format(buyer_wants)
    
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}])
    
    buyer_profile = response['choices'][0]['message']['content']
    print(" ")
    print("buyer_profile")
    print(buyer_profile)
    
    prompt = """
    You are an AI assistant. Your job is write a description of buyer property that writes a rich natural language paragraph based on the structured input provided.
    
    Structured Input:
    {}
    
    Desired Output:
    "The buyer is looking for a..."
    
    """.format(buyer_profile)
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}])
    buyer_profile_desc = response['choices'][0]['message']['content']
    buyer_embedding = embedder.encode([buyer_profile_desc]).tolist()[0]
    
    results = collection.query(
        query_embeddings=[buyer_embedding],
        n_results=5, 
        include=["documents", "distances"]
    )
        
    property_pick = results['documents'][0]

    return property_pick

In [218]:
property_pick = get_property_picks_listingdb(buyer_wants, collection, embedder)
[print('\n#######'+i) for i in property_pick]

 
buyer_profile
{
    "size": "condo",
    "property_type": "condo",
    "important_features": ["balcony", "modern kitchen"],
    "amenities": ["lots of coffee shops"],
    "transportation": "city centre",
    "urban_level": "high",
    "neighborhood_vibes": "writing",
    "architectural style": "modern",
    "pool": "not specified"
}

#######Neighborhood: Downtown Chic Condos
Price: $950,000
Bedrooms: 3
Bathrooms: 2.5
Property: Condo
View: City skyline
Size: 1,800 sqft

Description: Live in style in this modern 3-bedroom, 2.5-bathroom condo located in the heart of the vibrant Downtown Chic Condos community. With floor-to-ceiling windows offering breathtaking views of the city skyline, this condo exudes urban sophistication. The open-concept living space features designer finishes, a gourmet kitchen with sleek appliances, and a private balcony for outdoor enjoyment. The luxurious master suite includes a spa-like bathroom and walk-in closet, creating a peaceful retreat in the midst of t

[None, None, None, None, None]

### Step 6: Personalizing Listing Descriptions

- LLM Augmentation: For each retrieved listing, use the LLM to augment the description, tailoring it to resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for.
- Maintaining Factual Integrity: Ensure that the augmentation process enhances the appeal of the listing without altering factual information.

In [219]:
def get_personalized_listings(buyer_wants, property_pick):
    prompt = """
    You are an AI assistant realestate agent. Your job is to re-write the property listings' Description in a way that highlights aspects most relevant to the buyer’s preferences.
    Ensure personalization emphasizes characteristics appealing to the buyer without omitting or altering factual information about the property.
    Personalized listing should include all output fields: "Neighborhood","Price","Bedrooms","Bathrooms","Property", "House", "Pool","Size", "Description", "Neighborhood Description".
    
    Buyer preferences: {}
    
    Property Listings: {}
    
    Desired output:
    Return a JSON array. Each element must be an object with two keys:
    - "original": the original property listing.
    - "personalized": the rewritten version tailored to the buyer's preferences.
    
    Example format:
    [
      {{
        "original": "Original listing text here.",
        "personalized": "Personalized version here."
      }},
    ]
    
    
    """.format(buyer_wants, '\n\n'.join(property_pick))
    
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}])
    
    return response['choices'][0]['message']['content']

In [220]:
personalized_picks = get_personalized_listings(buyer_wants, property_pick)
import json

data = json.loads(personalized_picks)

In [221]:
data[1]

{'original': {'Neighborhood': 'Urban Chich Lofts',
  'Price': '$800,000',
  'Bedrooms': 2,
  'Bathrooms': 2,
  'Property': 'Condo',
  'View': 'City skyline',
  'Size': '1,200 sqft',
  'Description': 'Immerse yourself in the vibrant city life in this stylish urban loft located in the trendy Urban Chic Lofts community. This 2-bedroom, 2-bathroom condo offers panoramic views of the city skyline from large windows, creating a modern and sophisticated ambiance. The open-concept living space features high ceilings, exposed brick walls, and sleek finishes, perfect for contemporary urban living. The gourmet kitchen boasts stainless steel appliances and a large island, ideal for hosting intimate gatherings. With a rooftop terrace for al fresco dining and entertainment, this loft is the epitome of chic city living.',
  'Neighborhood Description': 'Urban Chic Lofts is a sought-after urban community with easy access to restaurants, bars, art galleries, and boutiques for the ultimate city living ex

### Step 7: Deliverables and Testing

- Test your "HomeMatch" application and make sure it meets all of the requirements in the rubric(opens in a new tab). Your project code will be run when it's assessed. Enter different "buyer preferences" and ensure it works.
- Jupyter Notebook/Python Program: Compile the application code in a Jupyter notebook or a standalone Python program. Ensure the code is well-commented and logically structured.
- Example Outputs: Include example outputs showcasing how user preferences are processed and how the application generates personalized listing descriptions. You can include these in comments in your application or in a Jupyter notebook that's saved with outputs.

In [222]:
def get_question_answers(previous_example):
    prompt = """
    You are a propsective home buyer answering questions. Please answers the following questions 
    with that background in mind. Be unique from previous answers provided and choose between whether you want a house or condo:

    Background:
    
    questions = [
        "Please, tell me what you are looking for in a property, such as number of bedrooms, bathrooms, location, and other specific requirements, and I will try to find you a match!",
        "Is there anything else very important about the property I should consider, such as architecture or features?",
        "Is there anything else very important about the neighborhood I should consider?"
    ]

    Previous Example: {}
    
    Desired output:
    Python List, Each element must be a string answering the questions above. Do not include number answers in response.  
    
    """.format(previous_example)
    
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}])
    
    return response['choices'][0]['message']['content']

previous_example= ['I am looking for a house with at least 3 bedrooms, 2 bathrooms, preferably in a quiet suburban area with good schools nearby.', 
                   'I would also like a backyard for gardening and a garage for storage. It is very important to me that the property has a modern and open floor plan, with plenty of natural light. I also prefer a home with energy-efficient appliances.',
                   'It is important that the neighborhood is safe, with low crime rates and friendly neighbors. Access to parks, walking trails, and convenient shopping centers would be a plus.']

buyer_wants = get_question_answers(previous_example)
print(buyer_wants)

[
    "I am looking for a condo with 2 bedrooms and 2 bathrooms, ideally located in a bustling urban area close to restaurants and entertainment venues.",
    "I would also like a property with a modern minimalist architecture and sleek finishes. It is important to me that the condo has smart home features and a balcony with a city view.",
    "It is important that the neighborhood is vibrant with a lively atmosphere, good public transportation options, and access to cultural events and attractions."
]


### Example 1

In [223]:
buyer_wants = get_question_answers(buyer_wants)
property_pick = get_property_picks_listingdb(buyer_wants, collection, embedder)
personalized_picks = get_personalized_listings(buyer_wants, property_pick)


 
buyer_profile
{
    "size": "3 bedrooms",
    "property_type": "house",
    "important_features": ["wraparound porch", "fireplace", "spacious backyard"],
    "amenities": ["good schools", "parks nearby", "grocery stores", "local shops"],
    "transportation": "not specified",
    "urban_level": "suburban",
    "neighborhood_vibes": "quiet",
    "architectural style": "not specified",
    "pool": "not specified"
}


In [224]:
print(buyer_wants)

```
"I am looking for a house with 3 bedrooms and 2 bathrooms, preferably in a quiet suburban neighborhood with good schools and parks nearby."
"I really love houses with a wraparound porch and a fireplace. It would be great if the property has a spacious backyard for gardening and outdoor activities."
"I value neighborhoods that have a strong sense of community, low crime rates, and easy access to grocery stores and local shops."
```


In [225]:
print(personalized_picks)

[
    {
        "original": {
            "Neighborhood": "City View Heights",
            "Price": "$950,000",
            "Bedrooms": 5,
            "Bathrooms": 3.5,
            "Property": "House",
            "View": "City skyline views",
            "Size": "3,000 sqft",
            "Description": "Experience city living at its finest in this contemporary 5-bedroom, 3.5-bathroom home located in the prestigious City View Heights neighborhood. With panoramic city skyline views from the rooftop terrace, this home is perfect for entertaining guests and enjoying the urban landscape. The sleek and modern kitchen features stainless steel appliances and quartz countertops, while the spacious master suite offers a luxurious retreat with a spa-like bathroom. With a landscaped backyard and outdoor dining area, this home offers the perfect blend of urban sophistication and comfort.",
            "Neighborhood Description": "City View Heights is a vibrant urban community with easy access to t

### Example 2

In [226]:
buyer_wants = get_question_answers(buyer_wants)
property_pick = get_property_picks_listingdb(buyer_wants, collection, embedder)
personalized_picks = get_personalized_listings(buyer_wants, property_pick)

 
buyer_profile
{
      "size": "2 bedrooms and 2 bathrooms",
      "property_type": "condo",
      "important_features": ["vibrant urban area", "trendy restaurants", "cafes nearby"],
      "amenities": ["rooftop terrace", "gym facilities"],
      "transportation": "walkable neighborhood",
      "urban_level": "high",
      "neighborhood_vibes": "variety of entertainment options, safe for living alone",
      "architectural style": "modern",
      "pool": "not specified"
    }


In [227]:
print(buyer_wants)

"I am looking for a condo with 2 bedrooms and 2 bathrooms, located in a vibrant urban area with trendy restaurants and cafes nearby."
"I am particularly interested in condos with modern architecture and amenities such as a rooftop terrace or gym facilities."
"I prioritize neighborhoods that are walkable, have a variety of entertainment options, and are safe for living alone."


In [228]:
print(personalized_picks)

[
  {
    "original": {
      "Neighborhood": "Downtown Chic Condos",
      "Price": "$950,000",
      "Bedrooms": 3,
      "Bathrooms": 2.5,
      "Property": "Condo",
      "View": "City skyline",
      "Size": "1,800 sqft",
      "Description": "Live in style in this modern 3-bedroom, 2.5-bathroom condo located in the heart of the vibrant Downtown Chic Condos community. With floor-to-ceiling windows offering breathtaking views of the city skyline, this condo exudes urban sophistication. The open-concept living space features designer finishes, a gourmet kitchen with sleek appliances, and a private balcony for outdoor enjoyment. The luxurious master suite includes a spa-like bathroom and walk-in closet, creating a peaceful retreat in the midst of the bustling city.",
      "Neighborhood Description": "Downtown Chic Condos is a trendy urban community with easy access to renowned restaurants, art galleries, and entertainment venues. Residents can explore nearby parks, cultural landmark

### Step 8: Project Submission

- Generated Listings: Include a file that contains your synthetically generated real estate listings. Name this file "listings"
- Project Documentation: Include a readme file or an accompanying document explaining the functionality, how to run the code, and any prerequisites or dependencies.
- Code Submission: Submit the Jupyter Notebook or Python program on the "Project Submission Page" that follows the workspace page.